# Signature Validation Console
Interactive console for reviewing signature explainability, direction concordance, and recording validation decisions.



In [ ]:
import json
import os
import base64
import io
from typing import List, Dict, Any

import requests
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML

# API configuration
API_BASE = os.environ.get("API_BASE", "http://localhost:8000/api/v1")
TIMEOUT = 10

# Widgets common outputs
status_output = widgets.Output()
explanation_output = widgets.Output()
download_output = widgets.Output()

# Simple helper to log to status panel

def log_status(message: str, color: str = "#333"):
    with status_output:
        print(f"[{color}] {message}")



In [ ]:
# Demo fallback data

demo_signatures = [
    {
        "id": "11111111-1111-1111-1111-111111111111",
        "name": "DEMO_SIG_A",
        "description": "Inflammation signature",
        "validation_status": "pending",
    },
    {
        "id": "22222222-2222-2222-2222-222222222222",
        "name": "DEMO_SIG_B",
        "description": "Metabolic stress signature",
        "validation_status": "approved",
    },
]

demo_datasets = [
    {"id": "aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa", "name": "Demo Dataset 1"},
    {"id": "bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb", "name": "Demo Dataset 2"},
]

demo_explanation = {
    "signature_id": demo_signatures[0]["id"],
    "signature_name": demo_signatures[0]["name"],
    "dataset_id": demo_datasets[0]["id"],
    "dataset_name": demo_datasets[0]["name"],
    "total_score": 1.32,
    "direction_concordance": 0.72,
    "all_contributions": [
        {"feature_name": "IL6", "weight": 1.0, "contribution": 0.42, "match_type": "exact", "direction_match": "match"},
        {"feature_name": "TNF", "weight": 1.0, "contribution": 0.31, "match_type": "exact", "direction_match": "match"},
        {"feature_name": "CRP", "weight": 0.8, "contribution": -0.15, "match_type": "fuzzy", "direction_match": "conflict"},
        {"feature_name": "IL1B", "weight": 0.9, "contribution": 0.27, "match_type": "exact", "direction_match": "match"},
        {"feature_name": "CXCL8", "weight": 0.7, "contribution": -0.05, "match_type": "fuzzy", "direction_match": "neutral"},
    ],
}



In [ ]:
def api_get(path: str, params: Dict[str, Any] | None = None):
    try:
        resp = requests.get(f"{API_BASE}{path}", params=params, timeout=TIMEOUT)
        resp.raise_for_status()
        return resp.json()
    except Exception as exc:  # noqa: BLE001
        log_status(f"API GET failed: {exc}", color="#d9534f")
        return None


def api_patch(path: str, payload: Dict[str, Any]):
    try:
        resp = requests.patch(f"{API_BASE}{path}", json=payload, timeout=TIMEOUT)
        resp.raise_for_status()
        return resp.json()
    except Exception as exc:  # noqa: BLE001
        log_status(f"API PATCH failed: {exc}", color="#d9534f")
        return None


def load_signatures():
    data = api_get("/signatures")
    if data is None:
        return demo_signatures
    return data


def load_datasets():
    data = api_get("/datasets")
    if data is None:
        return demo_datasets
    return data


def fetch_explanation(signature_id: str, dataset_id: str):
    data = api_get(f"/signatures/{signature_id}/explain/{dataset_id}")
    if data is None:
        return demo_explanation
    return data


def update_status(signature_id: str, status: str, reviewer_notes: str | None = None):
    payload = {"status": status}
    if reviewer_notes:
        payload["reviewer_notes"] = reviewer_notes
    return api_patch(f"/signatures/{signature_id}/status", payload)


def make_download_link(df: pd.DataFrame, filename: str = "contributions.csv"):
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)
    b64 = base64.b64encode(csv_buffer.getvalue().encode()).decode()
    return HTML(f"<a download='{filename}' href='data:text/csv;base64,{b64}'>⬇️ Download CSV</a>")



In [ ]:
# Load data
signatures = load_signatures()
datasets = load_datasets()

sig_options = {f"{s.get('name', 'Unknown')} ({s.get('validation_status', 'pending')})": s for s in signatures}
data_options = {d.get("name", "Dataset"): d for d in datasets}

signature_dropdown = widgets.Dropdown(options=list(sig_options.keys()), description="Signature")
dataset_dropdown = widgets.Dropdown(options=list(data_options.keys()), description="Dataset")
status_filter = widgets.Dropdown(options=["all", "pending", "approved", "rejected"], description="Status")
notes_area = widgets.Textarea(placeholder="Reviewer notes", description="Notes")
approve_btn = widgets.Button(description="Approve", button_style="success")
reject_btn = widgets.Button(description="Reject", button_style="danger")
refresh_btn = widgets.Button(description="Refresh", button_style="info")

controls = widgets.VBox([
    widgets.HBox([signature_dropdown, dataset_dropdown]),
    widgets.HBox([status_filter, refresh_btn]),
    notes_area,
    widgets.HBox([approve_btn, reject_btn]),
])

controls


In [ ]:
def lollipop_chart(contribs: List[Dict[str, Any]]):
    if not contribs:
        return go.Figure()
    df = pd.DataFrame(contribs)
    df = df.sort_values(by="contribution", key=lambda s: s.abs(), ascending=False)
    colors = ["#28a745" if v >= 0 else "#dc3545" for v in df["contribution"]]
    fig = go.Figure(
        go.Bar(
            x=df["contribution"],
            y=df["feature_name"],
            orientation="h",
            marker_color=colors,
            hovertemplate="%{y}: %{x:.3f}<extra></extra>",
        )
    )
    fig.update_layout(
        title="Feature Contributions",
        height=max(300, 30 * len(df)),
        margin=dict(l=120, r=40, t=50, b=40),
        showlegend=False,
    )
    return fig


def direction_concordance_bar(contribs: List[Dict[str, Any]]):
    if not contribs:
        return go.Figure()
    total = len(contribs)
    match = sum(1 for c in contribs if c.get("direction_match") == "match")
    neutral = sum(1 for c in contribs if c.get("direction_match") in {"neutral", "unknown"})
    conflict = total - match - neutral
    segments = {
        "Match": (match / total) if total else 0,
        "Neutral": (neutral / total) if total else 0,
        "Conflict": (conflict / total) if total else 0,
    }
    fig = go.Figure()
    cum = 0
    colors = {"Match": "#28a745", "Neutral": "#6c757d", "Conflict": "#dc3545"}
    for label, frac in segments.items():
        fig.add_trace(
            go.Bar(
                x=[frac * 100],
                y=["Direction Concordance"],
                orientation="h",
                marker_color=colors[label],
                name=label,
                hovertemplate=f"{label}: %{x:.1f}%<extra></extra>",
            )
        )
        cum += frac
    fig.update_layout(barmode="stack", height=80, margin=dict(l=10, r=10, t=10, b=10), showlegend=True)
    return fig



In [ ]:
def build_contrib_table(contribs: List[Dict[str, Any]]):
    if not contribs:
        return pd.DataFrame()
    df = pd.DataFrame(contribs)
    df = df[["feature_name", "match_type", "direction_match", "weight", "contribution"]]
    def light(dm):
        return {"match": "🟢", "neutral": "🟡"}.get(dm, "🔴")
    df["traffic"] = df["direction_match"].apply(light)
    df["contribution"] = df["contribution"].round(3)
    return df


def render_explanation():
    explanation_output.clear_output()
    download_output.clear_output()
    selected_sig = sig_options.get(signature_dropdown.value)
    selected_ds = data_options.get(dataset_dropdown.value)
    if not selected_sig or not selected_ds:
        with explanation_output:
            print("Please select signature and dataset")
        return

    data = fetch_explanation(selected_sig["id"], selected_ds["id"])
    contribs = data.get("all_contributions", [])

    fig = lollipop_chart(contribs)
    conc_fig = direction_concordance_bar(contribs)
    table_df = build_contrib_table(contribs)

    with explanation_output:
        display(widgets.HTML(f"<h3>{data.get('signature_name')} vs {data.get('dataset_name')}</h3>"))
        display(widgets.HTML(f"<b>Total Score:</b> {data.get('total_score', 0):.2f}"))
        display(widgets.HTML(f"<b>Direction Concordance:</b> {data.get('direction_concordance', 0):.0%}"))
        display(fig)
        display(conc_fig)
        if not table_df.empty:
            display(table_df)
            display(make_download_link(table_df))
        else:
            print("No contributions available")



In [ ]:
def on_refresh(_):
    global signatures, datasets, sig_options, data_options
    signatures = load_signatures()
    datasets = load_datasets()
    if status_filter.value != "all":
        signatures = [s for s in signatures if s.get("validation_status", "pending") == status_filter.value]
    sig_options = {f"{s.get('name', 'Unknown')} ({s.get('validation_status', 'pending')})": s for s in signatures}
    data_options = {d.get("name", "Dataset"): d for d in datasets}
    signature_dropdown.options = list(sig_options.keys())
    dataset_dropdown.options = list(data_options.keys())
    log_status("Lists refreshed", color="#0c5460")


def on_approve(_):
    _update_status("approved")


def on_reject(_):
    _update_status("rejected")


def _update_status(status: str):
    selected_sig = sig_options.get(signature_dropdown.value)
    if not selected_sig:
        log_status("Select a signature first", color="#d9534f")
        return
    resp = update_status(selected_sig["id"], status=status, reviewer_notes=notes_area.value.strip() or None)
    if resp is None:
        log_status("Status update failed (using demo mode?)", color="#d9534f")
    else:
        log_status(f"Status updated to {status}", color="#28a745")
    on_refresh(None)
    render_explanation()


refresh_btn.on_click(on_refresh)
approve_btn.on_click(on_approve)
reject_btn.on_click(on_reject)
signature_dropdown.observe(lambda change: render_explanation(), names="value")
dataset_dropdown.observe(lambda change: render_explanation(), names="value")
status_filter.observe(lambda change: on_refresh(None), names="value")

render_explanation()



In [ ]:
display(widgets.VBox([
    controls,
    widgets.Label("Status messages:"),
    status_output,
    widgets.Label("Explanation"),
    explanation_output,
    download_output,
]))

